In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import xgboost as xgb
import warnings
import yfinance as yf 
import pandas as pd

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")


ticker="AAPL"
    
    #나스닥 파일 불러오기
data_path = 'data/nasdq_20241104.csv'
data = pd.read_csv(data_path)

    
start_date = '2010-01-04'
end_date = '2024-10-25'
stock_data = yf.download(ticker, start=start_date, end=end_date)
    
    #AAPL 데이터 전처리
stock_data.reset_index(inplace=True)
stock_data['Date'] = stock_data['Date'].dt.strftime('%Y-%m-%d')
stock_data.columns = ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
stock_data.drop(columns=['Adj Close'], inplace=True)
    
    #Nasdaq 데이터 전처리
#global_data = data.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])

    #AAPL, Nasdaq merge
stock_merge_data = pd.merge(stock_data, data, on='Date', how='inner')

    #merge data 전처리
stock_merge_data['Date'] = pd.to_datetime(stock_merge_data['Date'])
stock_merge_data.set_index('Date', inplace=True)
stock_merge_data.columns = stock_merge_data.columns.str.replace(' ', '')

stock_merge_data['Close_InterestRate_Corr'] = stock_merge_data['Close'].rolling(252).corr(stock_merge_data['InterestRate'])
stock_merge_data['Close_VIX_Corr'] = stock_merge_data['Close'].rolling(252).corr(stock_merge_data['VIX'])
stock_merge_data['Rolling_Volatility'] = stock_merge_data['Close'].rolling(window=30).std()

stock_merge_data['Daily_Return'] = stock_merge_data['Close'].pct_change()
stock_merge_data['Volatility'] = stock_merge_data['Close'].rolling(window=30).std() # 이동 표준 편차를 계산하라
stock_merge_data['Rolling_Mean_Close'] = stock_merge_data['Close'].rolling(window=30).mean()
stock_merge_data.dropna(inplace=True)

stock_merge_data_head = stock_merge_data.iloc[:-10] # 앞쪽 
stock_merge_data_tail = stock_merge_data.iloc[-10:] # 뒷쪽 10개
    

X = stock_merge_data_head.drop(['Close', 'High', 'Low', 'Volume'], axis=1)  # Ensure 'Close' is dropped to create the feature set
y = stock_merge_data_head['Close']  # Target variable is 'Close' price



    # Step 1: Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Step 2: Check for NaN values and handle them
    # Using forward fill to handle NaN values (you can adjust this as needed)
X.fillna(method='ffill', inplace=True)

    # Step 3: Standardize the data
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

    # Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


   # Initialize and train a random forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf.predict(X_test)

ser_rf=pd.Series(y_pred_rf)


y_test

[*********************100%***********************]  1 of 1 completed


Date
2016-10-06     28.472500
2011-03-16     11.786071
2013-04-05     15.114286
2017-05-30     38.417500
2015-08-18     29.125000
                 ...    
2022-05-12    142.559998
2013-04-25     14.585000
2021-06-02    125.059998
2021-07-19    142.449997
2011-04-15     11.695000
Name: Close, Length: 691, dtype: float64

In [37]:
ser_rf=pd.Series(y_pred_rf)
ser_rf

0       28.570100
1       12.216068
2       15.255550
3       38.297674
4       28.834525
          ...    
686    140.377602
687     14.972096
688    125.005400
689    141.023901
690     11.934375
Length: 691, dtype: float64